In [51]:
from api import openligadb
import datetime

def extract_result(match_results):
    for match_result in match_results:
        if match_result['resultName'] == 'Endergebnis':
            return "finished", match_result['pointsTeam1'], match_result['pointsTeam2']
    return "future", None, None



def extract_winner(match_data):
    try:
        if match_data["goalsHome"] > match_data["goalsAway"]:
            return match_data["teamHomeId"]
        elif match_data["goalsHome"] < match_data["goalsAway"]:
            return match_data["teamAwayId"]
        else:
            return None
    except TypeError:
        return None


def extract_match_data(json):
    match_data = {}
    match_data["id"] = json["matchID"]
    match_data["date"] = datetime.datetime.strptime(json["matchDateTime"],"%Y-%m-%dT%H:%M:%S")
    match_data["teamHomeId"] = json["team1"]["teamId"]
    match_data["teamAwayId"] = json["team2"]["teamId"]
    match_data["status"], match_data["goalsHome"], match_data["goalsAway"] = extract_result(json["matchResults"])
    match_data["winnerTeamId"] = extract_winner(match_data)
    match_data["matchDay"] = json["group"]["groupOrderID"]
    return match_data

In [52]:
from tqdm import tqdm
import pickle
import pandas as pd

try:
    match_df = pickle.load(open("match_df.pck", "rb"))
except:
    match_data = []
    for year in tqdm(range(2005, 2024)):
        json = openligadb.get_all_season_matches("bl1", year)
        for match_data_json in json:
            match_data.append(extract_match_data(match_data_json))
    match_df = pd.DataFrame(match_data)
    pickle.dump(match_df, open("match_df.pck", "wb"))

100%|██████████| 19/19 [00:03<00:00,  4.87it/s]


In [53]:
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime

testing = False

if testing:
    testing_date = datetime.datetime.today() + relativedelta(days=-14)
    manip_df = match_df.copy(deep=True)
    manip_df = manip_df.loc[(match_df["status"] == "finished") & (match_df["date"] > testing_date)]
    manip_df["status"] = ["future" for _ in range(len(manip_df.index))]
    manip_df[["goalsHome", "goalsAway", "winnerTeamId"]] = np.NaN
    match_df[(match_df["status"] == "finished") & (match_df["date"] > testing_date)] = manip_df

In [ ]:
updatable_df = match_df[(match_df["status"] == "future") & (match_df["date"] < datetime.datetime.today())]
for _, row in tqdm(updatable_df.iterrows()):
    row_match_data = openligadb.get_match_data(row["id"])
    match_df.loc[row.name] = row_match_data
if len(updatable_df) > 0:
    pickle.dump(match_df, open("match_df.pck", "wb"))

In [ ]:
next_match_day = 1
for _, row in match_df[match_df["status"] == "future"].iterrows():
    next_match_day = row["matchDay"]
    if len(match_df[match_df["matchDay"] == next_match_day]) == 9:
        break
next_match_day_df = match_df[match_df["matchDay"] == next_match_day]